In this notebook, we will explore Netflix movies and TV shows with [kotlin/dataframe](https://github.com/Kotlin/dataframe). Also, we will use [lets-plot](https://github.com/JetBrains/lets-plot-kotlin) library for data visualization.

## Table of contents
* [**Imports**](attachment:./#Imports)
* [**Reading and first look**](attachment:./#Reading-and-first-look)
* [**TV Shows and Movies**](attachment:./#TV-Shows-and-Movies)
* [**Lifetimes and Release Times**](attachment:./#Lifetimes-and-Release-Times)
* [**Actors**](attachment:./#Actors)
* [**Countries**](attachment:./#Countries)
* [**Duration**](attachment:./#Duration)
* [**Ratings**](attachment:./#Ratings)

## Imports

We use the latest available versions of the libraries, the following line magic is responsible for this:

In [57]:
%useLatestDescriptors

Importing dataframe

In [58]:
%use dataframe

Importing the visualization library

In [59]:
%use lets-plot

We will also work with maps, so we need the corresponding libraries.

In [60]:
%use lets-plot-gt(gt="[23,)")
@file:DependsOn("org.geotools:gt-shapefile:[23,)")
@file:DependsOn("org.geotools:gt-cql:[23,)")

And write the necessary imports

In [61]:
import org.geotools.data.shapefile.ShapefileDataStoreFactory
import org.geotools.data.simple.SimpleFeatureCollection
import java.net.URL

## Reading and first look

To get started, need to read data from csv

In [62]:
val raw_df = DataFrame.read("netflix_titles.csv")

First look could be taken at its content

In [63]:
// taking a look at types and columns
raw_df.schema()

show_id: String
type: String
title: String
director: String?
cast: String?
country: String?
date_added: String?
release_year: Int
rating: String?
duration: String
listed_in: String
description: String


In [64]:
raw_df.size() // rowsCount x columnsCount

7787 x 12

In [65]:
raw_df.head() // return first five rows

DataFrame [5 x 12]

In [66]:
// Getting general statistics and info for each columns
raw_df.describe()

DataFrame [12 x 12]

Data consists of Netflix TV shows and movies up to 2020. Each row contains information about one specific project and consists of:
* `show_id` - unique show number
* `type` - ***TV Show*** or ***Movie***
* `title` - the name of a TV show or movie
* `director` - director's name
* `cast` - cast list
* `country` - the country where the title was released
* `date_added` - when the title was added on netflix
* `release_year` - the year the title was released
* `rating` - rating of the title
* `listed_in` - in which lists/genres the title is present on netflix
* `description` - title description

Before we get started, let's process the dataframe. It can be seen that `date_added` is of type `String`, let's [convert](https://kotlin.github.io/dataframe/convert.html) it to `LocalDate` for further convenience. Kotlin DataFrame provides built-in type converters for major types. We will use `String` -> `LocalDate` conversion and specify date format [pattern](https://docs.oracle.com/javase/8/docs/api/java/time/format/DateTimeFormatter.html)

In [67]:
val df = raw_df.dropNulls { date_added } // remove rows where `date_added` is not specified
        .convert { date_added }.toLocalDate("MMMM d, yyyy") // convert date_added to LocalDate using date pattern
// let's look at what type of column it turned out
df.date_added.type() 

Can't convert `August 14, 2020` to LocalDate
java.lang.IllegalStateException: Can't convert `August 14, 2020` to LocalDate
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: Can't convert `August 14, 2020` to LocalDate
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.ko

## TV Shows and Movies

First, let's see what more shows or films.

In [68]:
ggplot(raw_df.toMap()) +
    geomBar() { x="type"; fill="type" } +
    scaleFillManual(listOf("#00BCD4", "#009688")) +
    ggtitle("Count of TV Shows and Movies") +
    ggsize(900, 550)

It can be seen that the number of films on Netflix is about twice the number of TV shows. But has it always been this way? To do this, let's see if there was such a year when the number of TV Shows was more than Movies and let's see the cumulative amount for Movies and TV Shows.

In [69]:
val df_date_count = df.convert { date_added }.with { it.year } // converting `date_added` to extract `year`
  .groupBy { date_added } // grouping by `year` stored in `date_added`
  .aggregate { 
      count { type == "TV Show" } into "tvshows" // counting TV Shows into column `tvshows`
      count { type == "Movie" } into "movies" // counting Movies into column `movies`
  }
df_date_count

Line_89.jupyter-kts (3:57 - 61) Unresolved reference: year

Let's hold on and see how we can simplify this expression using more advanced operations. First of all, we can combine conversion of `date_added` into `year` and grouping using [`map`](https://kotlin.github.io/dataframe/map.html) function within [column selector](https://kotlin.github.io/dataframe/columnselectors.html).

In [70]:
val df_date_count = df.groupBy { date_added.map { it.year } } // grouping by year added extracted from `date_added`
  .aggregate { 
      count { type == "TV Show" } into "tvshows" // counting TV Shows into column `tvshows`
      count { type == "Movie" } into "movies" // counting Movies into column `movies`
  }

Line_90.jupyter-kts (3:54 - 58) Unresolved reference: year

Our [groupBy aggregation](https://kotlin.github.io/dataframe/groupby.html#aggregation) adds new columns for "TV Show" and "Movie". This is exactly what [`pivot`](https://kotlin.github.io/dataframe/pivot.html) does: generates new columns for every unique value in `type`.

In [71]:
df.groupBy { date_added.map { it.year } }
  .pivot { type }

Line_91.jupyter-kts (3:34 - 38) Unresolved reference: year

After `type` column is pivoted, we call [`aggregate`](https://kotlin.github.io/dataframe/pivot.html#aggregation) to specify metrics to be calculated for every data group.

In [72]:
df.groupBy { date_added.map { it.year } }
  .pivot { type }.aggregate { count () }

Line_92.jupyter-kts (3:34 - 38) Unresolved reference: year

Simple statistics can be aggregated without `aggregate`:

In [73]:
df.groupBy { date_added.map { it.year } }
  .pivot { type }.count()

Line_93.jupyter-kts (3:34 - 38) Unresolved reference: year

For `count` statistics there is even shorter API [`pivotCounts`](https://kotlin.github.io/dataframe/pivot.html#pivotcounts).

Here is the final version:

In [74]:
val df_date_count = df.groupBy { date_added.map { it.year } }.pivotCounts { type }
df_date_count

Line_94.jupyter-kts (3:54 - 58) Unresolved reference: year

Now we will prepare dataframe for rendering. We will call [`flatten`](https://kotlin.github.io/dataframe/flatten.html) to remove column grouping and convert dataframe to `Map`.

In [75]:
val data = df_date_count.flatten().toMap()

ggplot(data) +
    geomArea(color = "#BF360C", fill = "#BF360C", alpha = 0.5) { x="date_added"; y="TV Show" } +
    geomArea(color = "#01579B", fill = "#01579B", alpha = 0.5) { x="date_added"; y="Movie" } +
    theme(
        panelBackground = elementRect(color="#ECEFF1", fill = "#ECEFF1"),
        panelGrid = elementBlank(),
    ) +
    xlab("year") +
    ylab("count") +
    ggtitle("Number of titles by year") +
    ggsize(800, 500)

Line_95.jupyter-kts (3:12 - 25) Unresolved reference: df_date_count

It can be seen that more films were added every year than shows. Obviously, the cumulative sum of the movies was also always higher than the TV Shows, but let's build such a plot.

In [76]:
val df_cumsum_titles = df_date_count
    .sortBy { date_added } // sorting by date_added
    .cumSum { type.all() } // count cumulative sum for columns `TV Show` and `Movie` that are nested under column `type`
df_cumsum_titles

Line_96.jupyter-kts (3:24 - 37) Unresolved reference: df_date_count
Line_96.jupyter-kts (4:15 - 25) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.date_added: DataColumn<String?> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.date_added: String? defined in Line_10
Line_96.jupyter-kts (5:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10

In [77]:
ggplot(df_cumsum_titles.flatten().toMap()) +
    geomArea(color = "#BF360C", fill = "#BF360C", alpha = 0.5) { x="date_added"; y="TV Show" } +
    geomArea(color = "#01579B", fill = "#01579B", alpha = 0.5) { x="date_added"; y="Movie" } +
    theme(
        panelBackground = elementRect(color="#ECEFF1", fill = "#ECEFF1"),
        panelGrid = elementBlank(),
    ) +
    xlab("year") +
    ylab("cumulative count") +
    ggtitle("Cumulative count of titles by year") +
    ggsize(800, 500)

Line_97.jupyter-kts (3:8 - 24) Unresolved reference: df_cumsum_titles

## Lifetimes and Release Times

Let's take a look at the distribution by the lifetime of titles on the platform. To do this, find the most recently uploaded title and calculate the difference between the date it was added and the maximum date found.

In [78]:
import kotlinx.datetime.*

In [79]:
val maxDate = df.date_added.max()
val df_days = df.add {
    "days_on_platform" from { date_added.daysUntil(maxDate) } // adding column for number of days on the platform
    "months_on_platform" from { date_added.monthsUntil(maxDate) } // adding column for number of months on the platform
    "years_on_platform" from { date_added.yearsUntil(maxDate) } // // adding column for number of years on the platform
}

Line_99.jupyter-kts (5:42 - 51) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun Instant.daysUntil(other: Instant, timeZone: TimeZone): Int defined in kotlinx.datetime
public fun LocalDate.daysUntil(other: LocalDate): Int defined in kotlinx.datetime
Line_99.jupyter-kts (6:44 - 55) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun Instant.monthsUntil(other: Instant, timeZone: TimeZone): Int defined in kotlinx.datetime
public fun LocalDate.monthsUntil(other: LocalDate): Int defined in kotlinx.datetime
Line_99.jupyter-kts (7:43 - 53) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun Instant.yearsUntil(other: Instant, timeZone: TimeZone): Int defined in kotlinx.datetime
public fun LocalDate.yearsUntil(other: LocalDate): Int defined in kotlinx.datetime

In [80]:
val plot = ggplot(df_days["type", "days_on_platform"].toMap())
val p1 = plot + 
        geomHistogram(color="#ECEFF1", fill="#ef0b0b") { x="days_on_platform"; y="..density.." } +
        geomDensity(alpha=0.5, fill="#0befef") { x="days_on_platform" } +
        xlab("days") + ggtitle("Age distribution (in days) on Netflix")
val p2 = plot +
        geomBoxplot() { x="type"; y="days_on_platform"; fill="type" } +
        scaleFillManual(listOf("#792020", "#207979")) +
        ylab("days") + ggtitle("Boxplot for age (in days) by type")

GGBunch()
    .addPlot(p1, 0, 0, 500, 450)
    .addPlot(p2, 500, 0, 500, 450)

Line_100.jupyter-kts (3:19 - 26) Unresolved reference: df_days

The age distribution of titles on the platform is similar to movies and TV shows. But you can see in the second graph that there are very old titles among the movies compared to the shows. Let's take a closer look at this moment. To do this, let's build a graph of the duration in years of being on the platform of films and shows.

In [81]:
ggplot(df_days.select { type and years_on_platform }.toMap()) + // selecting two columns
    geomBar(position = Pos.dodge) { x = "years_on_platform"; fill = "type" } +
    scaleFillManual(listOf("#bc3076", "#30bc76")) +
    xlab("years") +
    ggtitle("Years of Movies and TV Shows on Netflix") +
    ggsize(900, 500)

Line_101.jupyter-kts (3:8 - 15) Unresolved reference: df_days
Line_101.jupyter-kts (3:25 - 29) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_101.jupyter-kts (3:34 - 51) Unresolved reference: years_on_platform

As you can see, movies are usually older than TV shows.
After that, you might ask yourself: how quickly were titles added to Netflix after their release? Well, finding the answer to it will be quite simple.

In [82]:
val df_years = df
                // adding a new column of the difference between the year of release and the year of addition
                .add("years_off_platform") {
                    date_added.year - release_year
                }
                // dropping negative values and equal to zero
                .filter { "years_off_platform"<Int>() > 0 }

Line_102.jupyter-kts (6:32 - 36) Unresolved reference: year

We dropped negative values because it happens that titles are added to the platform while it is still in production. Also dropped the zero values as they are of no interest. 

In [83]:
ggplot(df_years.select { years_off_platform }.toMap()) + 
    geomPoint(stat = Stat.count(), size = 7.5) { x="years_off_platform"; color="years_off_platform" } +
    scaleColorGradient(low = "#97a6d0", high = "#00256e") +
    theme().legendPosition(0.9, 0.83) +
    xlab("years") +
    ggtitle("How long does it take for a title to be added to Netflix?") +
    ggsize(1000, 500)

Line_103.jupyter-kts (3:8 - 16) Unresolved reference: df_years
Line_103.jupyter-kts (3:26 - 44) Unresolved reference: years_off_platform
Line_103.jupyter-kts (4:5 - 14) Cannot access class 'jetbrains.letsPlot.spatial.SpatialDataset'. Check your module classpath for missing or conflicting dependencies

Well, let's build the informal top charts for the oldest and newest movies and TV shows.

* ***Top 5 movies with the most days on Netflix***

In [84]:
// Top 5 oldest movies
df_days
    .filter { type == "Movie" } // filtering by type
    .sortByDesc { days_on_platform } // sorting by number of days on Netflix
    .select { type and title and country and date_added and release_year and duration } // selecting required columns
    .head() // taking first five rows

Line_104.jupyter-kts (4:1 - 8) Unresolved reference: df_days
Line_104.jupyter-kts (5:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_104.jupyter-kts (6:19 - 35) Unresolved reference: days_on_platform
Line_104.jupyter-kts (7:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_104.jupyter-kts (7:24 - 29) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.title: DataColumn<String> def

* ***Top 5 movies recently added on Netflix***

In [85]:
// Top 5 newest movies
df_days
    .filter { type == "Movie" }
    .sortBy { days_on_platform }
    .select { type and title and country and date_added and release_year and duration }
    .head()

Line_105.jupyter-kts (4:1 - 8) Unresolved reference: df_days
Line_105.jupyter-kts (5:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_105.jupyter-kts (6:15 - 31) Unresolved reference: days_on_platform
Line_105.jupyter-kts (7:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_105.jupyter-kts (7:24 - 29) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.title: DataColumn<String> def

* ***Top 5 TV Shows with most days on Netflix***

In [86]:
// Top 5 oldest shows
df_days
    .filter { type == "TV Show" }
    .sortByDesc { days_on_platform }
    .select { type and title and country and date_added and release_year and duration }
    .head()

Line_106.jupyter-kts (4:1 - 8) Unresolved reference: df_days
Line_106.jupyter-kts (5:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_106.jupyter-kts (6:19 - 35) Unresolved reference: days_on_platform
Line_106.jupyter-kts (7:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_106.jupyter-kts (7:24 - 29) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.title: DataColumn<String> def

* ***Top 5 TV Shows recently added on Netflix***

In [87]:
// Top 5 newest shows
df_days
    .filter { type == "TV Show" }
    .sortBy { days_on_platform }
    .select { type and title and country and date_added and release_year and duration }
    .head()

Line_107.jupyter-kts (4:1 - 8) Unresolved reference: df_days
Line_107.jupyter-kts (5:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_107.jupyter-kts (6:15 - 31) Unresolved reference: days_on_platform
Line_107.jupyter-kts (7:15 - 19) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.type: DataColumn<String> defined in Line_10
public final val DataRow<Line_10._DataFrameType>.type: String defined in Line_10
Line_107.jupyter-kts (7:24 - 29) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public final val ColumnsContainer<Line_10._DataFrameType>.title: DataColumn<String> def

You might be interested in what months are titles added most often?

In [88]:
val df_split_date = df
                    // splitting dates into four columns
                    .split { date_added }.by { listOf(it, it.dayOfWeek, it.month, it.year) }
                    .into("date", "day", "month", "year")
                    .sortBy("month") // sorting by month

Line_108.jupyter-kts (5:62 - 71) Unresolved reference: dayOfWeek
Line_108.jupyter-kts (5:76 - 81) Unresolved reference: month
Line_108.jupyter-kts (5:86 - 90) Unresolved reference: year

In [89]:
ggplot(
    df_split_date
        .groupBy { year and month }.count() // counting how many titles were added by year and month
        .convert { month }.toStr() // converting month to String for rendering
        .toMap()
    ) + 
    geomTile(height = 0.9, width = 0.9) { x = "year"; y="month"; fill="count" } +
    theme(panelBackground = elementBlank(),panelGrid = elementBlank()) +
    scaleFillGradient(low = "#FFF3E0", high = "#E65100") +
    ggtitle("Content additions by month and year") +
    ggsize(900, 700)

Line_109.jupyter-kts (4:5 - 18) Unresolved reference: df_split_date
Line_109.jupyter-kts (5:20 - 24) Unresolved reference: year
Line_109.jupyter-kts (5:29 - 34) Unresolved reference: month
Line_109.jupyter-kts (6:20 - 25) Unresolved reference: month

## Actors

In this section, let's take a look at the actors and directors who make the content. First, let's determine the average number of actors in titles.

In [90]:
// splitting cast and couting number of actors
val cast_df = df
    .split { cast }.by(',').inplace()
    .add("size_cast") { "cast"<List<String>>().size }


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


In [91]:
ggplot(cast_df.toMap()) +
    geomHistogram(bins = 50, showLegend = false) { x = "size_cast"; fill="..count.." } +
    scaleFillGradient(low = "#E0F7FA", high = "#006064") +
    xlab("actors") +
    ggtitle("Number of people on cast") +
    ggsize(950, 650)


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


It can be seen that usually 8-9 people are included in the cast.

But what about who exactly is involved in creating the content? Let's take a look at these actors and how many times they took part in movies and shows.

In [92]:
// counting the participation of each actor
val actors_df = cast_df.cast.explode().valueCounts()
actors_df

Line_112.jupyter-kts (4:30 - 37) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun <T> DataColumn<Collection<TypeVariable(T)>>.explode(): DataColumn<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataColumn<DataFrame<TypeVariable(T)>>.explode(): ColumnGroup<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.explode(vararg columns: String, dropEmpty: Boolean = ...): DataFrame<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T, C> DataFrame<TypeVariable(T)>.explode(vararg columns: KProperty<TypeVariable(C)>, dropEmpty: Boolean = ...): DataFrame<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T, C> DataFrame<TypeVariable(T)>.explode(vararg columns: ColumnReference<TypeVariable(C)>, dropEmpty: Boolean = ...): DataFrame<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
pub

In [93]:
ggplot(actors_df.toMap()) +
    geomHistogram(
        stat = Stat.identity, sampling = samplingPick(30), color = "#E8F5E9", boundary = 1.0, showLegend = false
    ) { x="cast"; y="count"; fill="cast" } +
    coordFlip() +
    scaleFillHue() +
    xlab("cast") +
    ggtitle("Top 30 actors") +
    ggsize(950, 900)

Line_113.jupyter-kts (3:8 - 17) Unresolved reference: actors_df

Anupam Kher is definitely in the lead with 42 titles. Now we will split the castes for participation in movies or shows.

In [94]:
val actors = cast_df.pivot { type }.aggregate {
    cast.explode().valueCounts()
}
actors

Line_114.jupyter-kts (4:10 - 17) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun <T> DataColumn<Collection<TypeVariable(T)>>.explode(): DataColumn<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataColumn<DataFrame<TypeVariable(T)>>.explode(): ColumnGroup<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.explode(vararg columns: String, dropEmpty: Boolean = ...): DataFrame<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T, C> DataFrame<TypeVariable(T)>.explode(vararg columns: KProperty<TypeVariable(C)>, dropEmpty: Boolean = ...): DataFrame<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T, C> DataFrame<TypeVariable(T)>.explode(vararg columns: ColumnReference<TypeVariable(C)>, dropEmpty: Boolean = ...): DataFrame<TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
pub

In [95]:
val p1 = ggplot(actors.`TV Show`.toMap()) +
            geomHistogram(
              stat = Stat.identity, sampling = samplingPick(30), color = "#E8F5E9", boundary = 1.0, showLegend = false
            ) { x="cast"; y="count"; fill="cast" } +
             coordFlip() +
            scaleFillGradient(low = "#263238", high = "#ECEFF1") +
            xlab("cast") +
            ggtitle("Top 30 actors in Shows")

val p2 = ggplot(actors.Movie.toMap()) +
            geomHistogram(
              stat = Stat.identity, sampling = samplingPick(30), color = "#E8F5E9", boundary = 1.0, showLegend = false
            ) { x="cast"; y="count"; fill="cast" } +
             coordFlip() +
            scaleFillGradient(low = "#006064", high = "#E0F7FA") +
            xlab("cast") +
            ggtitle("Top 30 actors in Movies")

GGBunch()
    .addPlot(p1, 0, 0, 500, 700)
    .addPlot(p2, 500, 0, 500, 700)

Line_115.jupyter-kts (3:17 - 23) Unresolved reference: actors
Line_115.jupyter-kts (12:17 - 23) Unresolved reference: actors

How about directors? Let's see the top 10 directors with more appearence on Netflix catalog.

In [96]:
val directors_df = df.valueCounts { director }


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


In [97]:
ggplot(directors_df.toMap()) +
    geomHistogram(
            stat = Stat.identity, sampling = samplingPick(10), color = "#E8F5E9", boundary = 1.0, showLegend = false
        ) { x = "director"; y = "count"; fill = "director"} +
        scaleFillHue() +
        coordFlip() +
        xlab("Name") +
        ggtitle("Top 10 directors") +
        ggsize(850, 500)


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


These people work very productively.

## Countries

Here we will examine the distribution of content by country.

We need to do some small manipulations to render map.

In [98]:
val factory = ShapefileDataStoreFactory()

In [99]:
val worldFeatures : SimpleFeatureCollection = with("naturalearth_lowres") {
    val url = "https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/shp/${this}/${this}.shp"
    factory.createDataStore(URL(url)).featureSource.features
}

// Convert Feature Collection to SpatialDataset.
// Use 10 decimals to encode floating point numbers (this is the default).
val world = worldFeatures.toSpatialDataset(10)
val voidTheme = theme().axisLineBlank().axisTextBlank().axisTicksBlank().axisTitleBlank()
val worldLimits = coordMap(ylim = -55 to 85)

Line_119.jupyter-kts (10:27 - 43) Cannot access class 'jetbrains.letsPlot.spatial.SpatialDataset'. Check your module classpath for missing or conflicting dependencies
Line_119.jupyter-kts (11:25 - 38) 'axisLineBlank(): theme' is deprecated. Use: theme(axisLine="blank"
Line_119.jupyter-kts (11:41 - 54) 'axisTextBlank(): theme' is deprecated. Use: theme(axisText="blank"
Line_119.jupyter-kts (11:57 - 71) 'axisTicksBlank(): theme' is deprecated. Use: theme(axisTicks="blank"
Line_119.jupyter-kts (11:74 - 88) 'axisTitleBlank(): theme' is deprecated. Use: theme(axisTitle="blank"

Let's add another dataframe with country labels.

In [100]:
val countries = DataFrame.readCSV("country_codes.csv")
countries.head()

DataFrame [5 x 5]

In [101]:
// counting number of titles by county and joining them with country codes dataframe
val df_country = df.valueCounts { country }.join(countries)
df_country


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


In [102]:
ggplot() +
    geomMap(data = df_country.toMap(), map = world, mapJoin = "iso" to "iso_a3", color = "white") { fill="count" } +
    scaleFillGradient(low = "#FFF3E0", high = "#E65100", name = "Number of Titles") +
    ggsize(1000, 800) + voidTheme + worldLimits

Line_124.jupyter-kts (4:5 - 12) Cannot access class 'jetbrains.letsPlot.spatial.SpatialDataset'. Check your module classpath for missing or conflicting dependencies
Line_124.jupyter-kts (4:46 - 51) Unresolved reference: world
Line_124.jupyter-kts (6:25 - 34) Unresolved reference: voidTheme
Line_124.jupyter-kts (6:37 - 48) Unresolved reference: worldLimits

The map clearly shows where the content is mainly produced and gets to Netflix. Let's take a closer look at the top of such countries.

In [103]:
ggplot(df_country[0..9].sortByDesc { count }.toMap()) +
    geomBar(stat = Stat.identity, fill = "#00796B") { x="country"; y="count" } +
    ggtitle("Top 10 Countries") +
    ggsize(900, 450)

Line_125.jupyter-kts (3:36 - 45) Type mismatch: inferred type is Unit but ColumnSet<TypeVariable(C)> was expected
Line_125.jupyter-kts (3:38 - 43) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public val ColumnsContainer<ColumnDescription>.count: DataColumn<Int> defined in org.jetbrains.kotlinx.dataframe.api
public val ColumnsContainer<ValueCount>.count: DataColumn<Int> defined in org.jetbrains.kotlinx.dataframe.api
public val DataRow<ColumnDescription>.count: Int defined in org.jetbrains.kotlinx.dataframe.api
public val DataRow<ValueCount>.count: Int defined in org.jetbrains.kotlinx.dataframe.api

## Duration

How long does the content usually last to keep the viewer?

In [104]:
val df_dur = df
                .split { duration }.by(" ").inward("duration_num", "duration_scale") // splitting duration by time and scale inward
                .convert { "duration"["duration_num"] }.toInt() // converting by column path
                .update { "duration"["duration_scale"] }.with { if (it == "Seasons") "Season" else it }
df_dur.head()


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


In [105]:
val durations = df_dur.pivot { type }.values { duration }
durations


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


In [106]:
val p1 = ggplot(durations.Movie.toMap()) { x="duration_num" } + 
            geomHistogram(bins = 100, color="#e0f7fa", fill="#00BCD4") { y="..density.." } +
            geomDensity(alpha=1, size =1, color = "#d41900") + xlab("minutes") +
            ggtitle("Distribution of movies duration in minutes")

val p2 = ggplot(durations.`TV Show`.toMap()) { x = "duration_num" } + statCount(fill = "#00BCD4") + xlab("seasons") + 
            ggtitle("Number of seasons of tv show")

GGBunch()
    .addPlot(p1, 0, 0, 1000, 500)
    .addPlot(p2, 0, 500, 1000, 500)

Line_128.jupyter-kts (3:27 - 32) Unresolved reference: Movie
Line_128.jupyter-kts (8:27 - 36) Unresolved reference: `TV Show`

And according to tradition, the top longest movies and TV shows.

* ***Top 5 movies with highest duration***

In [107]:
df_dur.xs("Movie") { type }
    .sortByDesc { duration.duration_num }.head()
    .select { title and country and date_added and release_year and duration.all() }

Line_129.jupyter-kts (4:17 - 42) Type mismatch: inferred type is Unit but ColumnSet<TypeVariable(C)> was expected
Line_129.jupyter-kts (4:28 - 40) Unresolved reference: duration_num

* ***Top 5 TV shows with most seasons***

In [108]:
df_dur.xs("TV Show") { type }
    .sortByDesc { duration.duration_num }.head()
    .select { title and country and date_added and release_year and duration.all() }

Line_130.jupyter-kts (4:17 - 42) Type mismatch: inferred type is Unit but ColumnSet<TypeVariable(C)> was expected
Line_130.jupyter-kts (4:28 - 40) Unresolved reference: duration_num

And in the top content producing countries, how long are movies and TV shows?

In [109]:
val list_top_countries = df_country.country.take(10).toSet()

val df_cntr = df_dur
    .filter { country in list_top_countries }
    .pivot { type }.aggregate { 
        groupBy{ country }.mean { duration.duration_num }
    }
df_cntr

Line_131.jupyter-kts (8:33 - 58) Type mismatch: inferred type is Unit but ColumnSet<TypeVariable(C)?> was expected
Line_131.jupyter-kts (8:44 - 56) Unresolved reference: duration_num

In [110]:
val p1 = ggplot(df_cntr.Movie.toMap()) +
        geomHistogram(
            stat = Stat.identity, boundary = 1.0, color = 0xE0F7FA, showLegend = false
        ) { 
            x = "country";
            y = "duration_num";
            fill = "duration_num";
        } +
        scaleFillGradient(low = "#ECEFF1", high = "#263238") +
        ylab("Minute") +
        xlab("Name") +
        ggtitle("Top 30 cast on Movies")
        
val p2 = ggplot(df_cntr.`TV Show`.toMap()) +
        geomHistogram(
            stat = Stat.identity, boundary = 1.0, color = 0xE0F7FA, showLegend = false
        ) { 
            x = "country";
            y = "duration_num";
            fill = "duration_num"
        } +
        scaleFillGradient(low = "#E0F7FA", high = "#006064") +
        ylab("Season") +
        xlab("Name") +
        ggtitle("Top 30 cast on TV Shows")

GGBunch()
    .addPlot(p1, 0, 0, 900, 550)
    .addPlot(p2, 0, 550, 900, 550)

Line_132.jupyter-kts (3:17 - 24) Unresolved reference: df_cntr
Line_132.jupyter-kts (16:17 - 24) Unresolved reference: df_cntr

## Ratings

Finally, let's take a look at the rating column.
Here we will find out what is the most commonly assigned rating for films and shows.

In [111]:
ggplot(df.toMap()) +
        geomHistogram(boundary = 1.0, color = 0xE0F7FA, showLegend = false) { x = "rating"; fill="rating" } +
        scaleFillHue() +
        xlab("Rating") +
        ggtitle("Rating of Titles") +
        ggsize(950, 500)


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)


In [112]:
ggplot(df.toMap()) + 
    geomBar(position=Pos.dodge) {x="rating"; fill="type"} + 
    scaleFillManual(listOf("#607D8B", "#00BCD4")) +
    xlab("Rating") +
    ggtitle("Rating of Titles") +
    ggsize(950, 500)


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)
